In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [3]:
data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [4]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()


In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [6]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [7]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [8]:
one_hot_labels = keras.utils.to_categorical(labels)

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

In [10]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10


F:\New folder\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 47ms/step - accuracy: 0.3827 - loss: 1.5133 - val_accuracy: 0.7459 - val_loss: 0.7803
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 19s 45ms/step - accuracy: 0.8852 - loss: 0.3741 - val_accuracy: 0.8259 - val_loss: 0.5496
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 45ms/step - accuracy: 0.9854 - loss: 0.0592 - val_accuracy: 0.8291 - val_loss: 0.5882
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 19s 47ms/step - accuracy: 0.9948 - loss: 0.0227 - val_accuracy: 0.8322 - val_loss: 0.6250
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 22s 50ms/step - accuracy: 0.9969 - loss: 0.0162 - val_accuracy: 0.8216 - val_loss: 0.6364
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 19s 47ms/step - accuracy: 0.9985 - loss: 0.0078 - val_accuracy: 0.8166 - val_loss: 0.6604
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 45ms/step - accuracy: 0.9980 - loss: 0.0089 - val_accuracy: 0.8128 - val_loss: 0.6849
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 45ms/step - accuracy: 0.9974 - loss: 0.0114 - val_accurac

In [24]:
input_text = "he is mad"
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
['anger']
